## 0. Setup
Import the Python modules used throughout the notebook. Make sure you have already installed the packages listed in the README (pandas, numpy, mutagen, unidecode).

### Package bootstrap
Install any missing Python packages required by this workflow so the import cell succeeds even on a fresh environment.

In [20]:
import importlib
import subprocess
import sys

REQUIRED_PACKAGES = {
    "pandas": "pandas",
    "numpy": "numpy",
    "mutagen": "mutagen",
    "unidecode": "Unidecode"
}

for module_name, install_name in REQUIRED_PACKAGES.items():
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Installing missing dependency: {install_name}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", install_name])
print("Dependency check complete.")

Dependency check complete.


In [21]:
from __future__ import annotations
from collections import defaultdict
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Iterable, List, Optional

import math
import re
import pandas as pd
import numpy as np
from mutagen import File as MutagenFile
from unidecode import unidecode

## 1. Configuration
Define key directories (relative to the repository root) and ensure the output folder for reports exists.

In [22]:
# Adjust these paths if you relocate folders.
REPO_ROOT = Path.cwd()
DOWNLOAD_ROOT = REPO_ROOT / "Downloaded"
SPOTIFY_PLAYLISTS = REPO_ROOT / "spotify_playlists"
SHOPPING_LIST_DIR = REPO_ROOT / "shopping_lists"
LIBRARY_INDEX_CSV = REPO_ROOT / "library_index.csv"

SHOPPING_LIST_DIR.mkdir(exist_ok=True)
print(f"Repository root: {REPO_ROOT}")
print(f"Download library: {DOWNLOAD_ROOT}")
print(f"Spotify playlist CSVs: {SPOTIFY_PLAYLISTS}")
print(f"Shopping/output directory: {SHOPPING_LIST_DIR}")

Repository root: c:\Users\CJ\Documents\GitHub\Exodusify
Download library: c:\Users\CJ\Documents\GitHub\Exodusify\Downloaded
Spotify playlist CSVs: c:\Users\CJ\Documents\GitHub\Exodusify\spotify_playlists
Shopping/output directory: c:\Users\CJ\Documents\GitHub\Exodusify\shopping_lists


## 2. Helper functions
Canonicalization helpers keep matching consistent between Spotify exports and local audio metadata.

In [23]:
NON_ALNUM = re.compile(r"[^a-z0-9]+")
FEAT_PATTERN = re.compile(r"\(feat\..*?\)", re.IGNORECASE)
REMIX_PATTERN = re.compile(r"-\s*(remaster(ed)?|remix|edit|mix).*", re.IGNORECASE)
AUDIO_EXTENSIONS = {'.mp3', '.flac', '.m4a', '.aac', '.ogg', '.wav', '.aiff'}

def canonicalize_string(value: Optional[str]) -> str:
    if not value:
        return ""
    normalized = unidecode(str(value))
    normalized = FEAT_PATTERN.sub("", normalized)
    normalized = REMIX_PATTERN.sub("", normalized)
    normalized = normalized.lower()
    normalized = NON_ALNUM.sub(" ", normalized)
    normalized = normalized.strip()
    return re.sub(r"\s+", " ", normalized)

def primary_artist(artists_field: Optional[str]) -> str:
    if not artists_field or not isinstance(artists_field, str):
        return ""
    first = artists_field.split(';')[0]
    return first.strip()

def friendly_playlist_name(csv_path: Path) -> str:
    name = csv_path.stem.replace('_', ' ')
    return name.strip()

def duration_ms_from_audio(audio_obj) -> Optional[int]:
    if audio_obj and audio_obj.info and getattr(audio_obj.info, 'length', None):
        return int(round(audio_obj.info.length * 1000))
    return None

## 3. Scan the downloaded library
Create or refresh an auditable `library_index.csv` capturing metadata for every audio file under `Downloaded/`.

In [24]:
def scan_downloaded_library(download_root: Path) -> pd.DataFrame:
    records = []
    if not download_root.exists():
        print(f"Download directory not found: {download_root}")
        return pd.DataFrame()

    for file_path in download_root.rglob('*'):
        if not file_path.is_file() or file_path.suffix.lower() not in AUDIO_EXTENSIONS:
            continue
        try:
            audio = MutagenFile(file_path)
        except Exception as exc:
            print(f"Failed to read {file_path}: {exc}")
            audio = None

        tags = getattr(audio, 'tags', None) if audio else None
        artist_tag = None
        title_tag = None
        album_tag = None

        if tags:
            artist_tag = tags.get('TPE1') or tags.get('artist')
            title_tag = tags.get('TIT2') or tags.get('title')
            album_tag = tags.get('TALB') or tags.get('album')

        artist_str = str(artist_tag.text[0]) if hasattr(artist_tag, 'text') else (artist_tag if isinstance(artist_tag, str) else None)
        title_str = str(title_tag.text[0]) if hasattr(title_tag, 'text') else (title_tag if isinstance(title_tag, str) else None)
        album_str = str(album_tag.text[0]) if hasattr(album_tag, 'text') else (album_tag if isinstance(album_tag, str) else None)

        # Fallbacks from the path structure
        if not artist_str:
            artist_str = file_path.parent.name
        if not title_str:
            title_str = file_path.stem

        records.append({
            'file_path': file_path.relative_to(download_root).as_posix(),
            'artist_raw': artist_str,
            'title_raw': title_str,
            'album_raw': album_str,
            'artist_canonical': canonicalize_string(artist_str),
            'title_canonical': canonicalize_string(title_str),
            'duration_ms': duration_ms_from_audio(audio)
        })

    df = pd.DataFrame.from_records(records)
    if not df.empty:
        df.sort_values(['artist_canonical', 'title_canonical', 'file_path'], inplace=True)
    return df

library_index = scan_downloaded_library(DOWNLOAD_ROOT)
print(f"Indexed {len(library_index):,} local tracks")
if not library_index.empty:
    library_index.to_csv(LIBRARY_INDEX_CSV, index=False)
    display(library_index.head())
else:
    print('Library index is empty – check DOWNLOAD_ROOT or file extensions.')

Indexed 922 local tracks


,file_path,artist_raw,title_raw,album_raw,artist_canonical,title_canonical,duration_ms
3,3OH!3/Streets Of Gold/01 Beaumont.mp3,3OH!3,Beaumont,Streets Of Gold,3oh 3,beaumont,68467
11,3OH!3/WANT (Deluxe)/04 CHOKECHAIN.mp3,3OH!3,CHOKECHAIN,WANT (Deluxe),3oh 3,chokechain,211722
5,3OH!3/Streets Of Gold/04 Déjà Vu.mp3,3OH!3,Déjà Vu,Streets Of Gold,3oh 3,deja vu,184712
2,3OH!3/Now That's What I Call Music! 73/03 DONT...,3OH!3,DONTTRUSTME,Now That's What I Call Music! 73,3oh 3,donttrustme,192601
8,3OH!3/Streets Of Gold/10 Double Vision.mp3,3OH!3,Double Vision,Streets Of Gold,3oh 3,double vision,190746


## 4. Load Spotify playlist exports
Combine all CSV files in `spotify_playlists/` into a single DataFrame with helpful flags.

In [25]:
def load_spotify_playlists(csv_root: Path) -> pd.DataFrame:
    rows = []
    if not csv_root.exists():
        print(f"Spotify playlist directory not found: {csv_root}")
        return pd.DataFrame()

    csv_files = sorted(csv_root.glob('*.csv'))
    if not csv_files:
        print(f"No CSV files found in {csv_root}")
        return pd.DataFrame()

    for csv_file in csv_files:
        try:
            df = pd.read_csv(csv_file)
        except Exception as exc:
            print(f"Failed to read {csv_file}: {exc}")
            continue
        df['playlist_name'] = friendly_playlist_name(csv_file)
        df['is_liked'] = csv_file.name.lower() == 'liked_songs.csv'
        df['is_top_songs'] = csv_file.name.lower().startswith('your_top_songs_')
        rows.append(df)

    merged = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()
    if merged.empty:
        return merged

    merged['primary_artist'] = merged['Artist Name(s)'].apply(primary_artist)
    merged['artist_canonical'] = merged['primary_artist'].apply(canonicalize_string)
    merged['title_canonical'] = merged['Track Name'].apply(canonicalize_string)
    return merged

spotify_df = load_spotify_playlists(SPOTIFY_PLAYLISTS)
print(f"Loaded {len(spotify_df):,} Spotify rows across {spotify_df['playlist_name'].nunique() if not spotify_df.empty else 0} playlists")
if not spotify_df.empty:
    display(spotify_df.head())

Loaded 14,549 Spotify rows across 74 playlists



C:\Users\CJ\AppData\Local\Temp\ipykernel_30248\1881787435.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()


,Track URI,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Explicit,Added By,Added At,...,Liveness,Valence,Tempo,Time Signature,playlist_name,is_liked,is_top_songs,primary_artist,artist_canonical,title_canonical
0,spotify:track:5PHPENfE3RVmHGAA2A7Hfx,Sissy That Walk,Born Naked,RuPaul,2014-02-24,212693.0,1.0,False,22gilcer6gcjla3s2qjnnhgxy,2019-05-14T00:18:40Z,...,0.0865,0.591,126.046,4.0,2019 Second Chance Prom,False,False,RuPaul,rupaul,sissy that walk
1,spotify:track:7jman10UPhzhtOOqZLjSsh,Cover Girl,Champion,RuPaul,2009-04-07,178626.0,0.0,False,22gilcer6gcjla3s2qjnnhgxy,2019-05-13T19:33:24Z,...,0.0959,0.717,127.995,4.0,2019 Second Chance Prom,False,False,RuPaul,rupaul,cover girl
2,spotify:track:25Y5EIIljjyEhKdl23zv6j,Your Makeup Is Terrible,Anus,Alaska Thunderfuck,2015-06-23,237474.0,0.0,False,22gilcer6gcjla3s2qjnnhgxy,2019-05-13T19:36:45Z,...,0.0974,0.585,128.054,4.0,2019 Second Chance Prom,False,False,Alaska Thunderfuck,alaska thunderfuck,your makeup is terrible
3,spotify:track:2M2WJ7gBlcKNxdhyfPp9zY,Best of My Love,Rejoice,The Emotions,1977-06-10,220560.0,69.0,False,allyjclark-us,2019-05-13T19:40:48Z,...,0.0904,0.970,115.443,4.0,2019 Second Chance Prom,False,False,The Emotions,the emotions,best of my love
4,spotify:track:37Q5anxoGWYdRsyeXkkNoI,Heaven Is A Place On Earth,Greatest Vol.1 - Belinda Carlisle,Belinda Carlisle,1987,246520.0,27.0,False,allyjclark-us,2019-05-13T19:40:54Z,...,0.0497,0.793,122.902,4.0,2019 Second Chance Prom,False,False,Belinda Carlisle,belinda carlisle,heaven is a place on earth


## 5. Match Spotify tracks to the local library
Left-join on canonical artist/title keys and filter by duration tolerance where available.

In [26]:
DURATION_TOLERANCE_MS = 3000

def match_tracks(spotify_df: pd.DataFrame, library_df: pd.DataFrame, duration_tolerance_ms: int = DURATION_TOLERANCE_MS) -> pd.DataFrame:
    if spotify_df.empty:
        return pd.DataFrame()
    if library_df.empty:
        result = spotify_df.copy()
        result['file_path'] = pd.NA
        result['duration_ms_local'] = pd.NA
        return result

    lib_cols = library_df.rename(columns={'duration_ms': 'duration_ms_local'})
    merged = spotify_df.merge(lib_cols, how='left', on=['artist_canonical', 'title_canonical'], suffixes=('_spotify', '_local'))

    if 'duration_ms_local' in merged.columns:
        mask = merged['duration_ms_local'].notna() & merged['Duration (ms)'].notna()
        mismatched = mask & (merged['Duration (ms)'] - merged['duration_ms_local']).abs() > duration_tolerance_ms
        merged.loc[mismatched, ['file_path', 'duration_ms_local']] = pd.NA
    return merged

matched_df = match_tracks(spotify_df, library_index)
print(f"Matched rows: {len(matched_df):,}")
if not matched_df.empty:
    have_files = matched_df['file_path'].notna().sum()
    print(f"Tracks already downloaded: {have_files:,}")
    print(f"Tracks missing locally: {len(matched_df) - have_files:,}")
    display(matched_df.head())

Matched rows: 14,907
Tracks already downloaded: 3,046
Tracks missing locally: 11,861


,Track URI,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Explicit,Added By,Added At,...,is_liked,is_top_songs,primary_artist,artist_canonical,title_canonical,file_path,artist_raw,title_raw,album_raw,duration_ms_local
0,spotify:track:5PHPENfE3RVmHGAA2A7Hfx,Sissy That Walk,Born Naked,RuPaul,2014-02-24,212693.0,1.0,False,22gilcer6gcjla3s2qjnnhgxy,2019-05-14T00:18:40Z,...,False,False,RuPaul,rupaul,sissy that walk,NaN,NaN,NaN,NaN,NaN
1,spotify:track:7jman10UPhzhtOOqZLjSsh,Cover Girl,Champion,RuPaul,2009-04-07,178626.0,0.0,False,22gilcer6gcjla3s2qjnnhgxy,2019-05-13T19:33:24Z,...,False,False,RuPaul,rupaul,cover girl,NaN,NaN,NaN,NaN,NaN
2,spotify:track:25Y5EIIljjyEhKdl23zv6j,Your Makeup Is Terrible,Anus,Alaska Thunderfuck,2015-06-23,237474.0,0.0,False,22gilcer6gcjla3s2qjnnhgxy,2019-05-13T19:36:45Z,...,False,False,Alaska Thunderfuck,alaska thunderfuck,your makeup is terrible,NaN,NaN,NaN,NaN,NaN
3,spotify:track:2M2WJ7gBlcKNxdhyfPp9zY,Best of My Love,Rejoice,The Emotions,1977-06-10,220560.0,69.0,False,allyjclark-us,2019-05-13T19:40:48Z,...,False,False,The Emotions,the emotions,best of my love,NaN,NaN,NaN,NaN,NaN
4,spotify:track:37Q5anxoGWYdRsyeXkkNoI,Heaven Is A Place On Earth,Greatest Vol.1 - Belinda Carlisle,Belinda Carlisle,1987,246520.0,27.0,False,allyjclark-us,2019-05-13T19:40:54Z,...,False,False,Belinda Carlisle,belinda carlisle,heaven is a place on earth,NaN,NaN,NaN,NaN,NaN


## 6. Generate a dated shopping list
Aggregate missing tracks across playlists and export a timestamped CSV in `shopping_lists/`.

In [27]:
def build_shopping_list(matched_df: pd.DataFrame) -> pd.DataFrame:
    if matched_df.empty:
        return pd.DataFrame()
    missing = matched_df[matched_df['file_path'].isna()].copy()
    if missing.empty:
        return pd.DataFrame()

    grouped = (
        missing.groupby(['artist_canonical', 'title_canonical'], as_index=False)
        .agg({
            'primary_artist': 'first',
            'Track Name': 'first',
            'Album Name': lambda col: col.dropna().iloc[0] if col.dropna().any() else pd.NA,
            'Duration (ms)': 'first',
            'playlist_name': lambda col: sorted(set(col)),
            'is_liked': 'any',
            'is_top_songs': 'any'
        })
    )
    grouped['Playlists_Count'] = grouped['playlist_name'].apply(len)
    grouped['Playlists'] = grouped['playlist_name'].apply(lambda names: '; '.join(names))
    grouped.rename(columns={
        'primary_artist': 'Artist',
        'Track Name': 'Title',
        'Album Name': 'Album',
        'Duration (ms)': 'Duration_ms',
        'is_liked': 'Is_Liked',
        'is_top_songs': 'Is_Top_Songs'
    }, inplace=True)
    columns = ['Artist', 'Title', 'Album', 'Duration_ms', 'Playlists_Count', 'Playlists', 'Is_Liked', 'Is_Top_Songs']
    grouped = grouped[columns]
    grouped.sort_values(['Playlists_Count', 'Is_Liked', 'Artist', 'Title'], ascending=[False, False, True, True], inplace=True)
    return grouped

shopping_df = build_shopping_list(matched_df)
if shopping_df.empty:
    print('All playlist tracks already exist locally – no shopping list generated.')
else:
    timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    shopping_path = SHOPPING_LIST_DIR / f'shopping_list_{timestamp}.csv'
    shopping_df.to_csv(shopping_path, index=False)
    print(f"Shopping list saved to {shopping_path}")
    display(shopping_df.head())

Shopping list saved to c:\Users\CJ\Documents\GitHub\Exodusify\shopping_lists\shopping_list_2025-11-17-01-03-25.csv


,Artist,Title,Album,Duration_ms,Playlists_Count,Playlists,Is_Liked,Is_Top_Songs
2357,Glass Animals,Heat Waves,Dreamland,238805.0,15,2020s Dance; Liked Songs; Mega Hit Mix; Melodi...,True,True
5498,Point Break Candy,Hole In The Sun (feat. COS & Conway) [From Cyb...,Hole In The Sun (feat. COS & Conway) [From Cyb...,208066.0,11,2020s Dance; Cyberpunk 2077; Cyberpunk 2077 Ra...,True,True
6010,Sam Smith,Unholy (feat. Kim Petras),Unholy (feat. Kim Petras),156943.0,11,2020s Dance; Favorites; Liked Songs; Mega Hit ...,True,True
2526,Haezer,Dumb,Exit,98691.0,10,2020s Dance; Favorites; Liked Songs; New Favor...,True,True
3729,Kavinsky,Nightcall,Nightcall,258413.0,10,Cyberpunk 2077; Cyberpunk 2077 Radio; Cyberpun...,True,True


## 7. Generate an orphaned-tracks list
Highlight tracks that exist in `Downloaded/` but are not referenced by any current playlist snapshot.

In [28]:
def build_orphaned_tracks(matched_df: pd.DataFrame, library_df: pd.DataFrame) -> pd.DataFrame:
    if library_df.empty:
        return pd.DataFrame()
    playlist_keys = set(zip(matched_df['artist_canonical'], matched_df['title_canonical'])) if not matched_df.empty else set()
    library_df = library_df.copy()
    library_df['key'] = list(zip(library_df['artist_canonical'], library_df['title_canonical']))
    mask = library_df['key'].apply(lambda key: key not in playlist_keys)
    orphaned = library_df[mask].copy()
    if orphaned.empty:
        return pd.DataFrame()
    orphaned.rename(columns={
        'artist_raw': 'Artist',
        'title_raw': 'Title',
        'album_raw': 'Album',
        'duration_ms': 'Duration_ms'
    }, inplace=True)
    columns = ['Artist', 'Title', 'Album', 'Duration_ms', 'file_path']
    return orphaned[columns]

orphan_df = build_orphaned_tracks(matched_df, library_index)
if orphan_df.empty:
    print('No orphaned tracks – every local track appears in at least one playlist snapshot.')
else:
    timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    orphan_path = SHOPPING_LIST_DIR / f'orphaned_tracks_{timestamp}.csv'
    orphan_df.to_csv(orphan_path, index=False)
    print(f"Orphaned-track report saved to {orphan_path}")
    display(orphan_df.head())

Orphaned-track report saved to c:\Users\CJ\Documents\GitHub\Exodusify\shopping_lists\orphaned_tracks_2025-11-17-01-03-25.csv


,Artist,Title,Album,Duration_ms,file_path
14,6EJOU,I'm Not Your Mate,I'm Not Your Mate,316761,6EJOU/I'm Not Your Mate/01 I'm Not Your Mate.mp3
16,"753, Brecc",Dare - Brecc Remix,Dare EP,366367,"753, Brecc/Dare EP/03 Dare - Brecc Remix.mp3"
17,"A Boogie Wit da Hoodie, Kodak Black",Drowning (feat. Kodak Black),Drowning (feat. Kodak Black),209319,"A Boogie Wit da Hoodie, Kodak Black/Drowning (..."
49,"A Perfect Circle, James Iha",Blue - Bird Shake Mix,Complete Collection,237166,"A Perfect Circle, James Iha/Complete Collectio..."
48,"A Perfect Circle, James Iha",Outsider - Frosted Yogurt Mix,aMOTION,247562,"A Perfect Circle, James Iha/aMOTION/05 Outside..."


## 8. Show Playlist Statistics
Summarize key statistics about each playlist, including total tracks, matched tracks, missing tracks, and orphaned tracks.

In [30]:
if 'matched_df' not in globals():
    print("Run cells 1-7 to create 'matched_df' before generating playlist stats.")
elif matched_df.empty:
    print('Playlist DataFrame is empty – load Spotify CSVs first.')
else:
    stats = (
        matched_df
        .groupby('playlist_name', dropna=False)
        .agg(
            Total_Tracks=('Track Name', 'size'),
            Matched_Tracks=('file_path', lambda col: col.notna().sum()),
            Liked_Snapshot=('is_liked', 'any'),
            Top_Songs_Snapshot=('is_top_songs', 'any')
        )
        .reset_index()
    )
    stats['Missing_Tracks'] = stats['Total_Tracks'] - stats['Matched_Tracks']
    stats['Percent_Complete'] = (stats['Matched_Tracks'] / stats['Total_Tracks'] * 100).round(1)
    stats.sort_values(['Percent_Complete', 'playlist_name'], ascending=[False, True], inplace=True)

    overall_total = int(stats['Total_Tracks'].sum())
    overall_missing = int(stats['Missing_Tracks'].sum())
    overall_matched = overall_total - overall_missing

    missing_unique = (
        matched_df[matched_df['file_path'].isna()]
        .drop_duplicates(subset=['artist_canonical', 'title_canonical'])
        .shape[0]
    )

    print(
        f"Playlists analyzed: {len(stats)} | Tracks: {overall_total:,} | "
        f"Matched: {overall_matched:,} | Missing: {overall_missing:,}"
    )
    print(f"Unique missing tracks across all playlists: {missing_unique:,}")
    display(stats)


Playlists analyzed: 74 | Tracks: 14,907 | Matched: 3,046 | Missing: 11,861
Unique missing tracks across all playlists: 7,707


,playlist_name,Total_Tracks,Matched_Tracks,Liked_Snapshot,Top_Songs_Snapshot,Missing_Tracks,Percent_Complete
31,New Favorites,2047,961,False,False,1086,46.9
26,Liked Songs,2090,932,True,False,1158,44.6
14,Emo In 07,58,21,False,False,37,36.2
39,Power Pop,21,6,False,False,15,28.6
15,Favorites,3926,1010,False,False,2916,25.7
...,...,...,...,...,...,...,...
67,planet rave,160,0,False,False,160,0.0
70,vaporwave,100,0,False,False,100,0.0
71,🍄,11,0,False,False,11,0.0
72,🎄,20,0,False,False,20,0.0


## 9. Generate Playlists
Build on these DataFrames to generate Innioasis Y1 playlist files (`.m3u8`) containing all the real tracks.

In [31]:
PLAYLIST_EXPORT_DIR = REPO_ROOT / "generated_playlists"
PLAYLIST_RELATIVE_ROOT = Path("Music")  # Adjust if your device expects a different root folder.
PLAYLIST_EXPORT_DIR.mkdir(exist_ok=True)

if 'matched_df' not in globals():
    print("Run cells 1-7 to build 'matched_df' before exporting playlists.")
elif matched_df.empty:
    print('matched_df is empty – load Spotify CSVs and re-run matching first.')
else:
    def playlist_filename(name: str) -> str:
        safe = re.sub(r"[^A-Za-z0-9._-]+", "_", (name or "playlist").strip())
        safe = safe.strip('_') or "playlist"
        return f"{safe}.m3u8"

    exports = []
    grouped = matched_df.groupby('playlist_name', sort=False)
    for playlist_name, group in grouped:
        resolved = group[group['file_path'].notna()].copy()
        if resolved.empty:
            continue
        if 'Position' in resolved.columns:
            resolved.sort_values('Position', inplace=True)
        else:
            resolved = resolved.sort_index()

        playlist_path = PLAYLIST_EXPORT_DIR / playlist_filename(playlist_name)
        lines = ['#EXTM3U']
        for _, row in resolved.iterrows():
            duration_sec = int(round(row['Duration (ms)'] / 1000)) if pd.notna(row.get('Duration (ms)')) else -1
            artist = row.get('primary_artist') or row.get('Artist Name(s)') or 'Unknown Artist'
            title = row.get('Track Name') or 'Unknown Title'
            lines.append(f"#EXTINF:{duration_sec},{artist} - {title}")
            device_path = (PLAYLIST_RELATIVE_ROOT / Path(row['file_path'])).as_posix()
            lines.append(device_path)

        playlist_path.write_text('\n'.join(lines) + '\n', encoding='utf-8')
        exports.append({
            'playlist_name': playlist_name,
            'tracks_total': int(len(group)),
            'tracks_written': int(len(resolved)),
            'tracks_missing': int(group['file_path'].isna().sum()),
            'output_file': playlist_path.relative_to(REPO_ROOT).as_posix()
        })

    if not exports:
        print('No playlists had matched tracks – nothing exported.')
    else:
        summary = pd.DataFrame(exports)
        summary.sort_values('playlist_name', inplace=True)
        print(f"Exported {len(summary)} playlists to {PLAYLIST_EXPORT_DIR}")
        display(summary)


Exported 30 playlists to c:\Users\CJ\Documents\GitHub\Exodusify\generated_playlists


,playlist_name,tracks_total,tracks_written,tracks_missing,output_file
0,2019 Second Chance Prom,348,18,330,generated_playlists/2019_Second_Chance_Prom.m3u8
1,2020s Dance,25,1,24,generated_playlists/2020s_Dance.m3u8
2,Anthems,97,4,93,generated_playlists/Anthems.m3u8
3,Cosmic Love- second chance,78,6,72,generated_playlists/Cosmic_Love-_second_chance...
4,Emo In 07,58,21,37,generated_playlists/Emo_In_07.m3u8
5,Favorites,3926,1010,2916,generated_playlists/Favorites.m3u8
6,Florida,12,1,11,generated_playlists/Florida.m3u8
8,Gen Z Dance Party,79,4,75,generated_playlists/Gen_Z_Dance_Party.m3u8
9,HARD TECHNO,485,1,484,generated_playlists/HARD_TECHNO.m3u8
10,Liked Songs,2090,932,1158,generated_playlists/Liked_Songs.m3u8
